In [1]:
#Load libraries 

import pandas as pd
import sspa
import scipy
import numpy as np 
import os #get path location
import pickle

We make two different networks, one for the COVID cases 1-2 compared to COVID cases 3-7 <br>
This is because there are only 18 samples in common between the metabolomic and proteomic datasets

0       Common samples: 18           Metabolomic samples: 133        Proteomic samples: 123 <br>
1-2       Common samples: 45          Metabolomic samples: 45        Proteomic samples: 48 <br>
3-4       Common samples: 56          Metabolomic samples: 57        Proteomic samples: 59 <br>
5-7       Common samples: 27          Metabolomic samples: 28        Proteomic samples: 28 <br>

146 common samples overall,   128 cases (45 samples (WHO 1-2) vs 83 samples (WHO 3-7))

In [2]:
df = pd.read_csv('Data/Su_COVID_proteomics_processed_commoncases.csv', index_col=0)
df_mild = (df[df["WHO_status"] == '1-2']).iloc[:,:-2] #45 samples, remove the metadata
df_severe = (df[(df["WHO_status"] == '3-4') | (df["WHO_status"] == '5-7')]).iloc[:,:-2] #83 samples

Check that data is centred at zero (mean=zero and standard deviation=1 for each analyte):

In [4]:
df_num  = df.iloc[:,:-2] #all rows, all columns apart from last two
print(df_num.max().max())
print(df_num.min().min())
print(df_num.mean(axis = 0)) #mean of 0
print(df_num.std(axis = 0)) #sd of 1

7.157779188839509
-7.68875111551009
O00182    1.387779e-17
O00220   -2.255141e-17
O00253    3.469447e-17
O14763   -1.040834e-17
O14836    6.938894e-18
              ...     
Q9Y4K4   -5.551115e-17
Q9Y5A7    4.163336e-17
Q9Y5L3   -6.938894e-18
Q9Y5V3   -3.469447e-17
Q9Y653    7.979728e-17
Length: 454, dtype: float64
O00182    1.003929
O00220    1.003929
O00253    1.003929
O14763    1.003929
O14836    1.003929
            ...   
Q9Y4K4    1.003929
Q9Y5A7    1.003929
Q9Y5L3    1.003929
Q9Y5V3    1.003929
Q9Y653    1.003929
Length: 454, dtype: float64


In [5]:
df

,O00182,O00220,O00253,O14763,O14836,O43915,O94907,P00797,P01127,P01241,...,Q9ULX7,Q9UNK0,Q9Y478,Q9Y4K4,Q9Y5A7,Q9Y5L3,Q9Y5V3,Q9Y653,WHO_status,Group
sample_id,,,,,,,,,,,,,,,,,,,,,
INCOV001,0.662699,0.399579,-0.606434,0.253751,0.008918,-0.183710,-1.281445,1.504826,-0.656391,-0.728879,...,0.056692,-1.339019,-1.482133,-1.836151,-1.708529,-0.677567,-0.982971,1.703613,3-4,COVID19
INCOV002,0.823906,-0.852936,0.391248,0.131715,-0.334619,-0.645015,-0.485669,-0.119154,-0.252597,0.454615,...,-0.589103,-0.916950,-0.999042,-0.323778,-0.368824,-1.212782,-0.263024,-0.830373,5-7,COVID19
INCOV003,0.493235,0.326555,-0.197626,0.579083,-0.144856,-1.146117,-0.221334,1.816060,-0.066493,0.851057,...,-1.357965,-1.209485,-1.103151,-0.608847,-0.821175,-1.010808,-0.662941,1.026187,5-7,COVID19
INCOV004,0.670326,0.108866,0.855782,0.265247,2.005828,0.572044,-2.511053,-1.590256,-2.628240,-0.385150,...,0.286096,-1.880875,-1.514350,-1.686500,-1.523233,-0.877769,-1.305748,0.806740,3-4,COVID19
INCOV005,0.303574,0.408726,-0.042568,1.189616,-0.305316,-0.309161,-1.713525,0.222013,-1.703663,0.883956,...,-0.092095,-1.600513,-1.340302,-1.155597,-1.447396,0.293662,-0.783788,-0.612823,3-4,COVID19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INCOV135,-0.559506,-1.584543,-1.643940,-1.425504,-0.822713,-0.627537,-1.521368,-0.407370,-1.174002,-0.084159,...,1.412605,-0.208973,-0.638806,-0.789582,-0.693826,-0.374689,-1.150091,-1.031098,1-2,COVID19
INCOV136,-0.641303,-0.179435,-0.571055,-0.277352,-0.332874,-1.106487,1.130936,-0.231875,0.497005,-0.264566,...,0.796855,0.841765,1.359763,0.887098,1.239520,-0.831379,0.877162,1.989469,3-4,COVID19
INCOV137,0.279144,-0.405912,-0.443536,0.522554,0.816226,-0.859657,1.046241,1.352599,0.538445,-0.861205,...,0.518722,0.866393,0.878241,0.976843,1.191535,-0.619671,0.741753,-0.774813,5-7,COVID19


In [6]:
#Download the reactome pathways
reactome_pathways = sspa.process_reactome('Homo sapiens', infile = 'Data/UniProt2Reactome_all_Levels.txt', download_latest = False, filepath = None)

#Download the root pathways
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}
root_pathway_names = list(root_pathway_dict.keys())



### Step 1: Determine initial test-statistic

In [7]:
#Function to calculate the squared Spearman correlation matrix 

def squared_spearman_corr(data):
    kpca_scores = sspa.sspa_kpca(data, reactome_pathways)   
    kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns))) #using Sara's code to drop root pathways

    spearman_results = scipy.stats.spearmanr(kpca_scores)
    squared_spearman_coef = np.square(spearman_results[0]) #correlation coefficients (spearman_results[1] gives the p-values)

    return squared_spearman_coef,list(kpca_scores.columns)




#Function to calculate the absolute difference between two matrices and then determine the mean for each edge

def absolute_val(data1,data2,edgelist):
    abs_rho_squared = np.absolute(np.array(data1) - np.array(data2))

    #Mask the upper half of the dataframe (so I don't view the comparisons between the two same genes, and also the duplicate comparisons are removed)
    mask = abs_rho_squared.copy()
    mask = np.triu(np.ones(mask.shape)).astype(bool)
    mask = np.invert(mask) #invert true and false values so the diagonal is False as well
    non_dup_abs_squared = pd.DataFrame(abs_rho_squared, columns = edgelist, index = edgelist)
    non_dup_abs_squared = pd.DataFrame(non_dup_abs_squared).where(mask) #Replace all false values with NaN using mask

    abs_squared_list = non_dup_abs_squared.stack().reset_index()
    abs_squared_list['level_0'] = abs_squared_list["level_0"].astype(str) + ", " + abs_squared_list['level_1']
    abs_squared_list.columns = ["Edges","na","Initial_tstat"]
    abs_squared_list.index = abs_squared_list["Edges"]
    abs_squared_list = abs_squared_list.drop(columns = ["Edges","na"])

    return(abs_squared_list)

Note: For the delta squared correlation values for the unshuffled data (i.e. the real data) I keep the indices (pathway edges). Since I already have a record of the edges, there is no need to keep the edges for each permutation, since the order is the same each time. 

In [8]:
spearman_mild,edgelist = squared_spearman_corr(df_mild)
spearman_severe,edgelist = squared_spearman_corr(df_severe)

output = absolute_val(spearman_mild,spearman_severe,edgelist)

In [9]:
output

,Initial_tstat
Edges,
"R-HSA-109606, R-HSA-109581",0.214095
"R-HSA-109704, R-HSA-109581",0.442899
"R-HSA-109704, R-HSA-109606",0.308395
"R-HSA-110362, R-HSA-109581",0.091419
"R-HSA-110362, R-HSA-109606",0.028304
...,...
"R-HSA-983705, R-HSA-9824446",0.046766
"R-HSA-983705, R-HSA-983168",0.029076
"R-HSA-983705, R-HSA-983169",0.065720


In [11]:
output.to_csv("Data/permutation_test_files_proteomics/initial_tstats.csv")

### Step 2: Shuffle the labels

The sample labels are shuffled, rather than assigning the samples to two different groups (since the sizes of the 1-2 class with the 3-7 class is not equal). See HPC_permutation_script_metabolomic for more info.

### Step 3: Read in the permutation files 

Using the HPC, 10 files each store 10k permutations. 10 array jobs are carried out to read in all 10k values, make them absolute and count how many are above the initial test statistic. See permutation_distribution.ipynb for more info.

https://www.jwilber.me/permutationtest/# <br>
We take the absolute value for the absolute difference between the two groups to compute the initial test statistic. Each permutation calculates a new test statistic calculated from the ABSOLUTE difference between the two test groups. It makes no difference whether I take the absolute difference or regular difference between the two, since either way it's a two-tailed p-value distribution, just that with the absolute difference the negative values have been mapped to the positive side.

### Step 4: Compare the difference in edges with other networks 

Comparing the naive networks (mild vs severe) to see if they agree with the differential network results:

In [1]:
import networkx as nx

In [2]:
mild = nx.read_gml("Cytoscape/proteomic_final_mildcases.gml")
severe = nx.read_gml("Cytoscape/proteomic_final_severecases.gml")

In [3]:
mild_edges = list(mild.edges())
print(len(mild_edges))
severe_edges = list(severe.edges())
print(len(severe_edges))

8061
67986


In [4]:
#Edges present in the mild network but not the severe network
len(set(mild_edges) - set(severe_edges))

4357

In [5]:
#Edges present in the severe network but not the mild network
len(set(severe_edges) - set(mild_edges))

64282

In [6]:
#Reading in the edges expressed in the differential metabolomic network
with open('Data/permutation_test_files_proteomics/sig_edges.txt') as f:
    lines = f.readlines()

edges_remaining = []

edges = lines[0].split(",")

for index in range(0,len(edges),2):
    list1 = edges[index],(edges[index+1][1:])
    edges_remaining.append(tuple(list1))

print(len(edges_remaining))
edges_remaining

29327


[('R-HSA-111465', 'R-HSA-109581'),
 ('R-HSA-112314', 'R-HSA-109581'),
 ('R-HSA-112314', 'R-HSA-109606'),
 ('R-HSA-112315', 'R-HSA-109581'),
 ('R-HSA-114508', 'R-HSA-109581'),
 ('R-HSA-1168372', 'R-HSA-111465'),
 ('R-HSA-1168372', 'R-HSA-112314'),
 ('R-HSA-1168372', 'R-HSA-112315'),
 ('R-HSA-1168372', 'R-HSA-114508'),
 ('R-HSA-1169091', 'R-HSA-109606'),
 ('R-HSA-1169091', 'R-HSA-111465'),
 ('R-HSA-1169091', 'R-HSA-112314'),
 ('R-HSA-1169091', 'R-HSA-112315'),
 ('R-HSA-1169091', 'R-HSA-114508'),
 ('R-HSA-1226099', 'R-HSA-109581'),
 ('R-HSA-1226099', 'R-HSA-109606'),
 ('R-HSA-1226099', 'R-HSA-1168372'),
 ('R-HSA-1226099', 'R-HSA-1169091'),
 ('R-HSA-1227986', 'R-HSA-112314'),
 ('R-HSA-1227986', 'R-HSA-112315'),
 ('R-HSA-1227986', 'R-HSA-114508'),
 ('R-HSA-1227986', 'R-HSA-1169091'),
 ('R-HSA-1227990', 'R-HSA-112314'),
 ('R-HSA-1227990', 'R-HSA-112315'),
 ('R-HSA-1227990', 'R-HSA-114508'),
 ('R-HSA-1227990', 'R-HSA-1169091'),
 ('R-HSA-1234174', 'R-HSA-109581'),
 ('R-HSA-1234176', 'R-HSA-109

In [7]:
intersection = list(set(edges_remaining).intersection(list(set(mild_edges))))  
len(intersection) #659 edges out of 4,357

659

In [8]:
intersection = list(set(edges_remaining).intersection(list(set(severe_edges)))) 
len(intersection) #2517 edges out of 64,282

2517